# Quickstart with BentoML and XGBoost

Link to source code: https://github.com/bentoml/gallery/tree/main/xgboost

In [3]:
!pip install -r https://raw.githubusercontent.com/bentoml/gallery/main/xgboost/requirements.txt   --user

In [4]:
import xgboost
import bentoml

### Train a classifier using the agaricus dataset:

In [23]:
# read in data
dtrain = xgboost.DMatrix("data/agaricus.txt.train")

# specify parameters via map
param = {"max_depth": 2, "eta": 1, "objective": "binary:logistic"}
num_round = 2
bst = xgboost.train(param, dtrain, num_round)

In [24]:
bentoml.xgboost.save("agaricus", bst)

The "bentoml.xgboost.save" method is being deprecated. Use "bentoml.xgboost.save_model" instead


Model(tag="agaricus:ku24bnx7jsbt73c4", path="C:\Users\hwang\bentoml\models\agaricus\ku24bnx7jsbt73c4\")

In [7]:
!bentoml models list

 Tag                        Module              Size       Creation Time       
 agaricus:lz5x4d77jklwf3c4  bentoml.xgboost     2.73 KiB   2022-07-09 01:45:41 
 iris_clf:5fjpqlp7isa7j3c4  bentoml.sklearn     5.77 KiB   2022-07-09 01:06:37 
 iris_clf:gnq6orh7iomjl3c4  bentoml.sklearn     5.77 KiB   2022-07-09 00:54:23 
 iris_clf2:d3bdoqh7hom773���  bentoml.sklearn     5.77 KiB   2022-07-08 23:56:32 
 iris_clf:ecx4ls77hc2rh3c4  bentoml.sklearn     5.77 KiB   2022-07-08 23:35:07 
 iris_clf:kh3nvy74qwl5d3c4  bentoml.sklearn     5.77 KiB   2022-07-05 13:10:07 
 iris_clf:5vmudih4gcs7p3c4  bentoml.sklearn     5.77 KiB   2022-07-05 03:06:01 
 iris_clf:kanmpkp4fwp7r3c4  bentoml.sklearn     5.77 KiB   2022-07-05 02:40:08 
 iris_clf:prylms74fsl5f3c4  bentoml.sklearn     5.77 KiB   2022-07-05 02:34:13 
 iris_clf:sckamh74fcvkp3c4  bentoml.sklearn     5.77 KiB   2022-07-05 02:06:09 
 iris_clf:74p4xjp4e6qf73c4  bentoml.sklearn     5.77 KiB   2022-07-05 02:02:05 
 iris_clf:wjslp5x4ewnhl3c4  bentoml.sk

Test loading the model as a BentoML Runner instance:

# 주름버섯속 구분
https://ko.wikipedia.org/wiki/%EC%A3%BC%EB%A6%84%EB%B2%84%EC%84%AF%EC%86%8D


In [29]:
from dmatrix2np import dmatrix_to_numpy

converted_np_array = dmatrix_to_numpy(dtrain)
print(converted_np_array)

ModuleNotFoundError: No module named 'dmatrix2np'

In [32]:
# The "bentoml.sklearn.load_runner" method is being deprecated. Use `bentoml.sklearn.get("agaricus:latest").to_runner()` instead
# test_runner = bentoml.sklearn.load_runner("agaricus:latest")


test_runner = bentoml.xgboost.get("agaricus:latest").to_runner()
test_runner.run([0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1])  # => array(0.01241208, dtype=float32)

StateException: Runner is not initialized

In [18]:
help (test_runner)

Help on Runner in module bentoml._internal.runner.runner object:

class Runner(builtins.object)
 |  Runner(runnable_class: 't.Type[Runnable]', *, runnable_init_params: 't.Dict[str, t.Any] | None' = None, name: 'str | None' = None, scheduling_strategy: 't.Type[Strategy]' = <class 'bentoml._internal.runner.strategy.DefaultStrategy'>, models: 't.List[Model] | None' = None, max_batch_size: 'int | None' = None, max_latency_ms: 'int | None' = None, method_configs: 't.Dict[str, t.Dict[str, int]] | None' = None) -> 'None'
 |  
 |  Methods defined here:
 |  
 |  __attrs_init__(self, runnable_class: 't.Type[Runnable]', runnable_init_params: 'dict[str, t.Any]', name: 'str', models: 'list[Model]', resource_config: 'dict[str, t.Any]', runner_methods: 'list[RunnerMethod[t.Any, t.Any, t.Any]]', scheduling_strategy: 't.Type[Strategy]') -> None
 |      Method generated by attrs for class Runner.
 |  
 |  __delattr__ = _frozen_delattrs(self, name)
 |      Attached to frozen classes as __delattr__.
 |  


### Create a BentoML Service for serving the model

Note: using `%%writefile` here because `bentoml.Service` instance must be created in a separate `.py` file

In [19]:
%%writefile iris_classifier.py
import typing

import bentoml
import xgboost
from bentoml.io import NumpyNdarray, File

if typing.TYPE_CHECKING:
    import numpy as np

agaricus_runner = bentoml.xgboost.load_runner("agaricus:latest")

svc = bentoml.Service("agaricus", runners=[agaricus_runner])


@svc.api(input=NumpyNdarray(), output=NumpyNdarray())
def classify(input_data: "np.ndarray") -> "np.ndarray":
    return agaricus_runner.run(input_data)

Writing iris_classifier.py


Start a dev model server to test out the service defined above:

In [20]:
!bentoml serve agaricus.py:svc

^C


Open a new browser tab and test out the API endpoint from the web UI.

### Build a Bento for distribution and deployment

In [ ]:
bentoml.build(
    "agaricus.py:svc",
    include=["*.py"],
    exclude=["locustfile.py"]
    description="file:./README.md",
    python=dict(
        packages=["xgboost"]
    )
)

Starting a dev server with the Bento build:

In [ ]:
!bentoml serve agaricus:latest

## Hooray, your model is ready for production now 🎉

Bento is a standarized format for storing models alongside with all their API definitions, configuration, and environment settings. BentoML can start an REST API server serving a Bento, run a Bento as batch processing job on distributed dataset, or containerize all dependencies and models into a docker container image for easy production deployment.

### Optional: create a docker image for the model server

This will require docker to be installed and docker daemon to be running:

In [ ]:
bentoml.bentos.containerize("agaricus:latest")

## What's next?

* Learn more at http://docs.bentoml.org 📖
* Join [BentoML Slack community](https://join.slack.com/t/bentoml/shared_invite/enQtNjcyMTY3MjE4NTgzLTU3ZDc1MWM5MzQxMWQxMzJiNTc1MTJmMzYzMTYwMjQ0OGEwNDFmZDkzYWQxNzgxYWNhNjAxZjk4MzI4OGY1Yjg) 👈
* Follow us on [Twitter](https://twitter.com/bentomlai) 🐦
* Contribute to [BentoML on Github](https://github.com/bentoml/BentoML) 💻